# AI Memory Service - Advanced Analysis Notebook
## Comprehensive System Analysis and Optimization

This notebook provides advanced analysis tools for the AI Memory Service, including:
- System performance metrics
- Memory quality assessment
- Embedding analysis
- Search optimization
- Real-time monitoring

## 1. Environment Setup and Configuration

In [ ]:
# Import required libraries
import os
import sys
import json
import time
import asyncio
import requests
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Configuration
MEMORY_API_URL = os.getenv('MEMORY_API_URL', 'http://127.0.0.1:8080')
EMBEDDING_API_URL = os.getenv('EMBEDDING_API_URL', 'http://127.0.0.1:8090')
SIMILARITY_THRESHOLD = float(os.getenv('SIMILARITY_THRESHOLD', '0.1'))

print(f"Memory API: {MEMORY_API_URL}")
print(f"Embedding API: {EMBEDDING_API_URL}")
print(f"Similarity Threshold: {SIMILARITY_THRESHOLD}")

## 2. Service Health Check and Status

In [ ]:
class ServiceMonitor:
    """Monitor health and status of AI Memory services"""
    
    def __init__(self):
        self.services = {
            'memory': MEMORY_API_URL,
            'embedding': EMBEDDING_API_URL
        }
        self.health_history = []
    
    def check_service(self, name: str, url: str) -> Dict[str, Any]:
        """Check individual service health"""
        try:
            start = time.time()
            response = requests.get(f"{url}/health", timeout=5)
            latency = (time.time() - start) * 1000
            
            return {
                'name': name,
                'status': 'online' if response.status_code == 200 else 'degraded',
                'code': response.status_code,
                'latency_ms': round(latency, 2),
                'timestamp': datetime.now().isoformat()
            }
        except Exception as e:
            return {
                'name': name,
                'status': 'offline',
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }
    
    def check_all(self) -> pd.DataFrame:
        """Check all services"""
        results = []
        for name, url in self.services.items():
            result = self.check_service(name, url)
            results.append(result)
            self.health_history.append(result)
        
        df = pd.DataFrame(results)
        return df
    
    def visualize_health(self):
        """Visualize service health status"""
        df = self.check_all()
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        # Status pie chart
        status_counts = df['status'].value_counts()
        colors = {'online': 'green', 'degraded': 'orange', 'offline': 'red'}
        axes[0].pie(status_counts.values, labels=status_counts.index, 
                   autopct='%1.0f%%', colors=[colors.get(s, 'gray') for s in status_counts.index])
        axes[0].set_title('Service Status')
        
        # Latency bar chart
        online_services = df[df['status'] == 'online']
        if not online_services.empty:
            axes[1].bar(online_services['name'], online_services['latency_ms'])
            axes[1].set_ylabel('Latency (ms)')
            axes[1].set_title('Service Latency')
        
        plt.tight_layout()
        plt.show()
        
        return df

# Run health check
monitor = ServiceMonitor()
health_df = monitor.visualize_health()
display(health_df)

## 3. Memory System Analysis

In [ ]:
class MemoryAnalyzer:
    """Analyze AI Memory system performance and quality"""
    
    def __init__(self):
        self.api_url = MEMORY_API_URL
        self.memories = []
        self.stats = {}
    
    def fetch_memories(self, limit: int = 100) -> List[Dict]:
        """Fetch memories from the system"""
        try:
            response = requests.get(f"{self.api_url}/memories", 
                                  params={'limit': limit})
            if response.status_code == 200:
                self.memories = response.json()
                return self.memories
        except Exception as e:
            print(f"Error fetching memories: {e}")
        return []
    
    def analyze_distribution(self):
        """Analyze memory distribution and patterns"""
        if not self.memories:
            self.fetch_memories()
        
        # Convert to DataFrame for analysis
        df = pd.DataFrame(self.memories)
        
        # Calculate statistics
        self.stats = {
            'total_memories': len(df),
            'unique_ids': df['id'].nunique() if 'id' in df else 0,
            'avg_content_length': df['content'].str.len().mean() if 'content' in df else 0,
            'memory_types': df['type'].value_counts().to_dict() if 'type' in df else {},
            'date_range': {
                'earliest': df['timestamp'].min() if 'timestamp' in df else None,
                'latest': df['timestamp'].max() if 'timestamp' in df else None
            }
        }
        
        # Visualizations
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Memory types distribution
        if 'type' in df:
            df['type'].value_counts().plot(kind='bar', ax=axes[0, 0])
            axes[0, 0].set_title('Memory Types Distribution')
            axes[0, 0].set_xlabel('Type')
            axes[0, 0].set_ylabel('Count')
        
        # Content length distribution
        if 'content' in df:
            df['content'].str.len().hist(bins=30, ax=axes[0, 1])
            axes[0, 1].set_title('Content Length Distribution')
            axes[0, 1].set_xlabel('Length')
            axes[0, 1].set_ylabel('Frequency')
        
        # Temporal distribution
        if 'timestamp' in df:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df.set_index('timestamp').resample('D').size().plot(ax=axes[1, 0])
            axes[1, 0].set_title('Memories Over Time')
            axes[1, 0].set_xlabel('Date')
            axes[1, 0].set_ylabel('Count')
        
        # Similarity scores distribution (if available)
        if 'similarity' in df:
            df['similarity'].hist(bins=20, ax=axes[1, 1])
            axes[1, 1].set_title('Similarity Scores Distribution')
            axes[1, 1].set_xlabel('Similarity')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].axvline(x=SIMILARITY_THRESHOLD, color='r', 
                              linestyle='--', label=f'Threshold: {SIMILARITY_THRESHOLD}')
            axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()
        
        return self.stats

# Analyze memory system
analyzer = MemoryAnalyzer()
stats = analyzer.analyze_distribution()
print("\nMemory System Statistics:")
print(json.dumps(stats, indent=2, default=str))

## 4. Embedding Quality Assessment

In [ ]:
class EmbeddingTester:
    """Test and analyze embedding quality"""
    
    def __init__(self):
        self.embed_url = EMBEDDING_API_URL
        self.test_results = []
    
    def generate_embedding(self, text: str) -> Optional[np.ndarray]:
        """Generate embedding for text"""
        try:
            response = requests.post(
                f"{self.embed_url}/embed",
                json={'text': text}
            )
            if response.status_code == 200:
                return np.array(response.json()['embedding'])
        except Exception as e:
            print(f"Error generating embedding: {e}")
        return None
    
    def cosine_similarity(self, a: np.ndarray, b: np.ndarray) -> float:
        """Calculate cosine similarity between two vectors"""
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    def test_embedding_quality(self):
        """Test embedding quality with various text pairs"""
        test_pairs = [
            # Similar texts
            ("The cat sat on the mat", "A cat was sitting on a mat", "high"),
            ("Machine learning is powerful", "ML is a powerful technology", "high"),
            # Different texts
            ("The weather is sunny", "I love programming", "low"),
            ("Python is a programming language", "The ocean is deep", "low"),
            # Moderate similarity
            ("AI helps with automation", "Artificial intelligence automates tasks", "medium"),
            ("Data science uses statistics", "Statistical analysis in data", "medium")
        ]
        
        results = []
        for text1, text2, expected in test_pairs:
            emb1 = self.generate_embedding(text1)
            emb2 = self.generate_embedding(text2)
            
            if emb1 is not None and emb2 is not None:
                similarity = self.cosine_similarity(emb1, emb2)
                results.append({
                    'text1': text1[:30] + '...' if len(text1) > 30 else text1,
                    'text2': text2[:30] + '...' if len(text2) > 30 else text2,
                    'expected': expected,
                    'similarity': similarity,
                    'passed': self._check_expectation(similarity, expected)
                })
        
        self.test_results = pd.DataFrame(results)
        return self.test_results
    
    def _check_expectation(self, similarity: float, expected: str) -> bool:
        """Check if similarity meets expectation"""
        if expected == 'high':
            return similarity > 0.7
        elif expected == 'low':
            return similarity < 0.3
        else:  # medium
            return 0.3 <= similarity <= 0.7
    
    def visualize_results(self):
        """Visualize embedding test results"""
        if self.test_results.empty:
            self.test_embedding_quality()
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Similarity scores by expectation
        self.test_results.boxplot(column='similarity', by='expected', ax=axes[0])
        axes[0].set_title('Similarity Scores by Expected Category')
        axes[0].set_xlabel('Expected Similarity')
        axes[0].set_ylabel('Actual Similarity Score')
        
        # Pass/Fail distribution
        pass_counts = self.test_results['passed'].value_counts()
        axes[1].pie(pass_counts.values, labels=['Passed', 'Failed'], 
                   autopct='%1.0f%%', colors=['green', 'red'])
        axes[1].set_title('Test Results')
        
        plt.tight_layout()
        plt.show()
        
        return self.test_results

# Test embeddings
tester = EmbeddingTester()
test_results = tester.visualize_results()
display(test_results)

## 5. Search Performance Optimization

In [ ]:
class SearchOptimizer:
    """Optimize and analyze search performance"""
    
    def __init__(self):
        self.api_url = MEMORY_API_URL
        self.benchmark_results = []
    
    async def benchmark_search(self, query: str, threshold: float) -> Dict:
        """Benchmark a single search operation"""
        start = time.time()
        try:
            response = requests.post(
                f"{self.api_url}/search",
                json={'query': query, 'similarity_threshold': threshold}
            )
            latency = (time.time() - start) * 1000
            
            if response.status_code == 200:
                results = response.json()
                return {
                    'query': query,
                    'threshold': threshold,
                    'latency_ms': latency,
                    'results_count': len(results),
                    'status': 'success'
                }
        except Exception as e:
            return {
                'query': query,
                'threshold': threshold,
                'error': str(e),
                'status': 'failed'
            }
    
    async def run_benchmark_suite(self):
        """Run comprehensive search benchmarks"""
        queries = [
            "machine learning",
            "artificial intelligence",
            "data analysis",
            "neural networks",
            "optimization algorithms"
        ]
        
        thresholds = [0.1, 0.2, 0.3, 0.5, 0.7]
        
        tasks = []
        for query in queries:
            for threshold in thresholds:
                tasks.append(self.benchmark_search(query, threshold))
        
        results = await asyncio.gather(*tasks)
        self.benchmark_results = pd.DataFrame(results)
        return self.benchmark_results
    
    def analyze_performance(self):
        """Analyze search performance metrics"""
        if self.benchmark_results.empty:
            return None
        
        successful = self.benchmark_results[self.benchmark_results['status'] == 'success']
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Latency by threshold
        successful.groupby('threshold')['latency_ms'].mean().plot(
            kind='bar', ax=axes[0, 0]
        )
        axes[0, 0].set_title('Average Latency by Threshold')
        axes[0, 0].set_xlabel('Threshold')
        axes[0, 0].set_ylabel('Latency (ms)')
        
        # Results count by threshold
        successful.groupby('threshold')['results_count'].mean().plot(
            kind='line', marker='o', ax=axes[0, 1]
        )
        axes[0, 1].set_title('Average Results Count by Threshold')
        axes[0, 1].set_xlabel('Threshold')
        axes[0, 1].set_ylabel('Results Count')
        
        # Latency distribution
        successful['latency_ms'].hist(bins=20, ax=axes[1, 0])
        axes[1, 0].set_title('Latency Distribution')
        axes[1, 0].set_xlabel('Latency (ms)')
        axes[1, 0].set_ylabel('Frequency')
        
        # Success rate
        success_rate = (self.benchmark_results['status'] == 'success').mean() * 100
        axes[1, 1].text(0.5, 0.5, f'Success Rate\n{success_rate:.1f}%', 
                       fontsize=30, ha='center', va='center')
        axes[1, 1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Calculate optimal threshold
        optimal_threshold = self._find_optimal_threshold(successful)
        print(f"\nOptimal Threshold: {optimal_threshold}")
        
        return successful.describe()
    
    def _find_optimal_threshold(self, df: pd.DataFrame) -> float:
        """Find optimal threshold balancing results and performance"""
        # Score = normalized_results - normalized_latency
        by_threshold = df.groupby('threshold').agg({
            'results_count': 'mean',
            'latency_ms': 'mean'
        })
        
        # Normalize to 0-1 scale
        by_threshold['norm_results'] = (by_threshold['results_count'] - by_threshold['results_count'].min()) / \
                                       (by_threshold['results_count'].max() - by_threshold['results_count'].min())
        by_threshold['norm_latency'] = (by_threshold['latency_ms'] - by_threshold['latency_ms'].min()) / \
                                       (by_threshold['latency_ms'].max() - by_threshold['latency_ms'].min())
        
        # Calculate score (maximize results, minimize latency)
        by_threshold['score'] = by_threshold['norm_results'] - by_threshold['norm_latency']
        
        return by_threshold['score'].idxmax()

# Run search optimization
optimizer = SearchOptimizer()

# Run async benchmark
await optimizer.run_benchmark_suite()
performance_stats = optimizer.analyze_performance()
display(performance_stats)

## 6. Real-time System Dashboard

In [ ]:
class SystemDashboard:
    """Real-time monitoring dashboard"""
    
    def __init__(self):
        self.monitor = ServiceMonitor()
        self.running = False
    
    def create_dashboard(self):
        """Create interactive dashboard"""
        from IPython.display import display, HTML
        import ipywidgets as widgets
        
        # Create widgets
        self.status_output = widgets.Output()
        self.metrics_output = widgets.Output()
        self.log_output = widgets.Output(layout={'height': '200px', 'overflow': 'auto'})
        
        # Control buttons
        start_btn = widgets.Button(description="Start Monitoring")
        stop_btn = widgets.Button(description="Stop Monitoring")
        refresh_btn = widgets.Button(description="Refresh")
        
        start_btn.on_click(lambda b: self.start_monitoring())
        stop_btn.on_click(lambda b: self.stop_monitoring())
        refresh_btn.on_click(lambda b: self.update_dashboard())
        
        # Layout
        controls = widgets.HBox([start_btn, stop_btn, refresh_btn])
        dashboard = widgets.VBox([
            widgets.HTML("<h2>AI Memory Service Dashboard</h2>"),
            controls,
            widgets.HTML("<h3>Service Status</h3>"),
            self.status_output,
            widgets.HTML("<h3>System Metrics</h3>"),
            self.metrics_output,
            widgets.HTML("<h3>Activity Log</h3>"),
            self.log_output
        ])
        
        display(dashboard)
        self.update_dashboard()
    
    def update_dashboard(self):
        """Update dashboard with latest data"""
        with self.status_output:
            clear_output(wait=True)
            health = self.monitor.check_all()
            display(health.style.apply(self._style_status, axis=1))
        
        with self.metrics_output:
            clear_output(wait=True)
            metrics = self._get_system_metrics()
            display(pd.DataFrame([metrics]).T.rename(columns={0: 'Value'}))
        
        with self.log_output:
            self._log(f"Dashboard updated at {datetime.now().strftime('%H:%M:%S')}")
    
    def _style_status(self, row):
        """Style status rows based on health"""
        if row['status'] == 'online':
            return ['background-color: lightgreen'] * len(row)
        elif row['status'] == 'degraded':
            return ['background-color: yellow'] * len(row)
        else:
            return ['background-color: lightcoral'] * len(row)
    
    def _get_system_metrics(self) -> Dict:
        """Get current system metrics"""
        return {
            'Similarity Threshold': SIMILARITY_THRESHOLD,
            'Memory API': MEMORY_API_URL,
            'Embedding API': EMBEDDING_API_URL,
            'Health Checks': len(self.monitor.health_history),
            'Last Check': datetime.now().strftime('%H:%M:%S')
        }
    
    def _log(self, message: str):
        """Add message to log"""
        timestamp = datetime.now().strftime('%H:%M:%S')
        print(f"[{timestamp}] {message}")
    
    async def start_monitoring(self):
        """Start continuous monitoring"""
        self.running = True
        self._log("Monitoring started")
        
        while self.running:
            self.update_dashboard()
            await asyncio.sleep(5)  # Update every 5 seconds
    
    def stop_monitoring(self):
        """Stop monitoring"""
        self.running = False
        self._log("Monitoring stopped")

# Create dashboard
dashboard = SystemDashboard()
dashboard.create_dashboard()

## 7. Optimization Recommendations

In [ ]:
# Generate optimization recommendations based on analysis
def generate_recommendations():
    """Generate system optimization recommendations"""
    
    recommendations = []
    
    # Check similarity threshold
    if SIMILARITY_THRESHOLD > 0.3:
        recommendations.append({
            'category': 'Configuration',
            'priority': 'High',
            'issue': f'Similarity threshold too high ({SIMILARITY_THRESHOLD})',
            'recommendation': 'Reduce similarity threshold to 0.1-0.2 for 768D embeddings',
            'impact': 'Will significantly increase search recall'
        })
    
    # Check service health
    if monitor.health_history:
        offline_services = [h for h in monitor.health_history if h.get('status') == 'offline']
        if offline_services:
            recommendations.append({
                'category': 'Infrastructure',
                'priority': 'Critical',
                'issue': f'{len(offline_services)} service health checks failed',
                'recommendation': 'Start offline services or check network configuration',
                'impact': 'System cannot function without core services'
            })
    
    # Check embedding quality
    if not tester.test_results.empty:
        failed_tests = tester.test_results[~tester.test_results['passed']]
        if len(failed_tests) > 0:
            recommendations.append({
                'category': 'Model Quality',
                'priority': 'Medium',
                'issue': f'{len(failed_tests)} embedding tests failed',
                'recommendation': 'Review embedding model configuration and prompts',
                'impact': 'May affect search accuracy'
            })
    
    # Performance recommendations
    recommendations.extend([
        {
            'category': 'Performance',
            'priority': 'Medium',
            'issue': 'No caching layer detected',
            'recommendation': 'Implement Redis cache for frequent queries',
            'impact': 'Can reduce latency by 50-70%'
        },
        {
            'category': 'Scalability',
            'priority': 'Low',
            'issue': 'Single-instance deployment',
            'recommendation': 'Consider horizontal scaling with load balancer',
            'impact': 'Improve reliability and throughput'
        }
    ])
    
    # Display recommendations
    rec_df = pd.DataFrame(recommendations)
    
    # Style by priority
    def style_priority(val):
        colors = {
            'Critical': 'background-color: #ff4444; color: white',
            'High': 'background-color: #ff8844',
            'Medium': 'background-color: #ffdd44',
            'Low': 'background-color: #44ff44'
        }
        return colors.get(val, '')
    
    styled = rec_df.style.applymap(style_priority, subset=['priority'])
    
    print("\n📊 SYSTEM OPTIMIZATION RECOMMENDATIONS\n")
    display(styled)
    
    return rec_df

recommendations = generate_recommendations()

## 8. Export Analysis Results

In [ ]:
# Export all analysis results
def export_analysis():
    """Export analysis results to files"""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Create results directory
    results_dir = Path(f'analysis_results_{timestamp}')
    results_dir.mkdir(exist_ok=True)
    
    # Export data
    exports = {
        'health_status.csv': health_df,
        'memory_stats.json': stats,
        'embedding_tests.csv': tester.test_results,
        'search_benchmarks.csv': optimizer.benchmark_results,
        'recommendations.csv': recommendations
    }
    
    for filename, data in exports.items():
        filepath = results_dir / filename
        
        if isinstance(data, pd.DataFrame):
            data.to_csv(filepath, index=False)
        elif isinstance(data, dict):
            with open(filepath, 'w') as f:
                json.dump(data, f, indent=2, default=str)
        
        print(f"✅ Exported: {filepath}")
    
    # Generate summary report
    report = f"""
    AI Memory Service Analysis Report
    Generated: {datetime.now()}
    
    System Status:
    - Services Online: {len(health_df[health_df['status'] == 'online'])}/{len(health_df)}
    - Average Latency: {health_df[health_df['status'] == 'online']['latency_ms'].mean():.2f}ms
    
    Memory System:
    - Total Memories: {stats.get('total_memories', 0)}
    - Similarity Threshold: {SIMILARITY_THRESHOLD}
    
    Recommendations:
    - Critical: {len(recommendations[recommendations['priority'] == 'Critical'])}
    - High: {len(recommendations[recommendations['priority'] == 'High'])}
    - Medium: {len(recommendations[recommendations['priority'] == 'Medium'])}
    - Low: {len(recommendations[recommendations['priority'] == 'Low'])}
    """
    
    report_path = results_dir / 'summary_report.txt'
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"\n📁 Analysis results exported to: {results_dir}")
    print(report)

export_analysis()